In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.3.0


First, we'll create a simple dataset, and it's just a simple range containing 10 elements from zero to nine. We'll print each one out on its own line as you can see.

In [2]:
dataset = tf.data.Dataset.range(10)
for val in dataset:
   print(val.numpy())

0
1
2
3
4
5
6
7
8
9


Next, we'll window the data into chunks of five items, shifting by one each time. We'll see that this gives us the output of the first five items, and then the second five items, and then the third five items, etc. At the end of the dataset, when there isn't enough data to give us five items, you'll see shorter lines. 

In [4]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 
7 8 9 
8 9 
9 


To just get chunks of five records, we'll set `drop_reminder` to true. When we run it, we'll see that our data looks like this.

In [5]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


We've got even sets that are the same size. TensorFlow likes its data to be in numpy format. So we can convert it easily by calling the dot numpy method and when we print it, we can see it's now listed in square brackets. 

In [6]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
  print(window.numpy())


[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


Next up is to split into x's and y's or features and labels. We'll take the last column as the label, and we'll split using a lambda. We'll split the data into column minus one, which is all of the columns except the last one, and minus one column which is the last one only. Now we can see that we have a set of four items and a single item. Remember that the minus one column denotes the last value in the list, and column minus one denotes everything about the last value. As such, we can see zero, one, two, three and one, two, three, four before the split just for example.

In [7]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
for x,y in dataset:
  print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


Next of course, is to shuffle the data. This is achieved with the shuffle method. This helps us to rearrange the data so as not to accidentally introduce a sequence bias. Multiple runs will show the data in different arrangements because it gets shuffled randomly.

In [8]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
for x,y in dataset:
  print(x.numpy(), y.numpy())


[3 4 5 6] [7]
[2 3 4 5] [6]
[4 5 6 7] [8]
[0 1 2 3] [4]
[1 2 3 4] [5]
[5 6 7 8] [9]


 Finally, comes batching. By setting a batch size of two, our data gets batched into two x's and two y's at a time. For example, as we saw earlier, if x is zero, one, two, three, we can see that the corresponding y is four or if x is five, six, seven, eight, then our y is nine. So that's the workbook with the code that splits a data series into windows. Try it out for yourself, and once you're familiar with what it does, proceed to the next video. There you'll move to the seasonal dataset that you've been using two dates, and with this windowing technique, you'll see how to set up x's and y's that can be fed into a neural network to see how it performs with predicting values.

In [9]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)
for x,y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())


x =  [[1 2 3 4]
 [2 3 4 5]]
y =  [[5]
 [6]]
x =  [[3 4 5 6]
 [5 6 7 8]]
y =  [[7]
 [9]]
x =  [[4 5 6 7]
 [0 1 2 3]]
y =  [[8]
 [4]]
